# Group Assignment - Natural Language Processing
##### Dennis Depoi, Anne Szyprons, Leonie Schulz, Melanie Ullrich

## Pipeline

1. Einlesen der Daten
    - alle xml-Dateien zusammenführen
    - Beschränkung auf eine Wahlperiode 
2. Datentransformation
    - Datenbereinigung (Entfernen der überflüssigen Zeichen)
    - Bereinigung der Spalte "TEXT"
    - 3 Ansätze:
        - Personenerkennung mit NER mit Spacy und nltk hat nicht funktioniert 
        - Extrahierung der Teilnehmer der Sitzungen anhand des Namensverzeichnis
        - Verwendung der Stammdatendatei (xml)
        - manuelle Liste der Abgeordneten in der Wahlperiode
    - Erstellung eines Subdataframes (Name des Vortragenden, Unterbrechungen, etc (Präsi))
3. Datenverarbeitung
    - Case Conversion
    - Tokenisierung
    - Entfernen von Stopwörtern
    - Lemmatisierung
    
    
4. Anwendung von Methoden: 
- Anwendung von Statistik: Häufigkeitsberechnung (Wie oft wer unterbrochen?)
- bag of words - tf-idf Maß - was sind häufige Wörter?
- Clustering? Einteilung in Klassen nach Wörter (neutral, Zustimmung, Ablehung)


Erstellung TF-IDF-Matrix (sklearns TfidfVectorizer)
Clustering? Topic Modelling?



## To Do
- Bereinigung der Textzeilen
- if-Schleife erweitern für mega Daten (Teilnehmer)

# 1. Einlesen der Daten

In [1]:
#settings, imports & downloads
import pandas as pd
import os
from bs4 import BeautifulSoup
import re
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
import spacy

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
#! python -m spacy download de_core_news_md

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ulli\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Ulli\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Ulli\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Ulli\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
#settings
pd.options.display.max_colwidth = 200

In [3]:
# read xml files
folder_path = "data"
files_in_folder = os.listdir(folder_path)

result = []
for file in files_in_folder:
    soup = BeautifulSoup(open(folder_path + "/" + file, "r", encoding="utf-8"), "xml")
    d = {}
    for tag in soup.find_all(recursive=True):
        d[tag.name] = tag.get_text(strip=True)
        result.append(d)
        
#write in Dataframe
df = pd.DataFrame(result)
df = df.drop_duplicates()
df = df.drop(columns = "DOKUMENT") #delete redundant column
#df.to_csv("data_protokoll.csv") #write to csv file
df = df[20:30]
df

,WAHLPERIODE,DOKUMENTART,NR,DATUM,TITEL,TEXT
140,17,PLENARPROTOKOLL,17/22,10.02.2010,Plenarprotokoll vom 10.02.2010,"Plenarprotokoll 17/22\nDeutscher Bundestag\nStenografischer Bericht\n\n22. Sitzung\n\nBerlin, Mittwoch, den 10. Februar 2010\n\nI n h a l t :\nGlückwünsche zum Geburtstag des Abgeord-\nneten Leo D..."
147,17,PLENARPROTOKOLL,17/23,24.02.2010,Plenarprotokoll vom 24.02.2010,"Plenarprotokoll 17/23\nDeutscher Bundestag\nStenografischer Bericht\n\n23. Sitzung\n\nBerlin, Mittwoch, den 24. Februar 2010\n\nI n h a l t :\nTagesordnungspunkt 1:\n\nBefragung der Bundesergierun..."
154,17,PLENARPROTOKOLL,17/24,25.02.2010,Plenarprotokoll vom 25.02.2010,"Plenarprotokoll 17/24\nDeutscher Bundestag\nStenografischer Bericht\n\n24. Sitzung\n\nBerlin, Donnerstag, den 25. Februar 2010\n\nI n h a l t :\nErweiterung und Abwicklung der Tagesord-\nnung . ...."
161,17,PLENARPROTOKOLL,17/25,26.02.2010,Plenarprotokoll vom 26.02.2010,"Plenarprotokoll 17/25\nDeutscher Bundestag\nStenografischer Bericht\n\n25. Sitzung\n\nBerlin, Freitag, den 26. Februar 2010\n\nI n h a l t :\nTagesordnungspunkt 18:\n\n– Beschlussempfehlung und Be..."
168,17,PLENARPROTOKOLL,17/26,03.03.2010,Plenarprotokoll vom 03.03.2010,"Plenarprotokoll 17/26\nDeutscher Bundestag\nStenografischer Bericht\n\n26. Sitzung\n\nBerlin, Mittwoch, den 3. März 2010\n\nI n h a l t :\nTagesordnungspunkt 1:\nBefragung der Bundesregierung: 13...."
175,17,PLENARPROTOKOLL,17/27,04.03.2010,Plenarprotokoll vom 04.03.2010,"Plenarprotokoll 17/27\nDeutscher Bundestag\nStenografischer Bericht\n\n27. Sitzung\n\nBerlin, Donnerstag, den 4. März 2010\n\nI n h a l t :\nWahl der Abgeordneten Cornelia Behm als\nstellvertreten..."
182,17,PLENARPROTOKOLL,17/28,05.03.2010,Plenarprotokoll vom 05.03.2010,"Plenarprotokoll 17/28\nDeutscher Bundestag\nStenografischer Bericht\n\n28. Sitzung\n\nBerlin, Freitag, den 5. März 2010\n\nI n h a l t :\nGlückwünsche zum Geburtstag der Abgeord-\nneten Ulla Lötze..."
189,17,PLENARPROTOKOLL,17/29,16.03.2010,Plenarprotokoll vom 16.03.2010,"Plenarprotokoll 17/29\nDeutscher Bundestag\nStenografischer Bericht\n\n29. Sitzung\n\nBerlin, Dienstag, den 16. März 2010\n\nI n h a l t :\nGlückwünsche zum Geburtstag des Abgeord-\nneten Dr. Herb..."
196,17,PLENARPROTOKOLL,17/30,17.03.2010,Plenarprotokoll vom 17.03.2010,"Plenarprotokoll 17/30\nDeutscher Bundestag\nStenografischer Bericht\n\n30. Sitzung\n\nBerlin, Mittwoch, den 17. März 2010\n\nI n h a l t :\nTagesordnungspunkt I (Fortsetzung):\n\na) Zweite Beratun..."
203,17,PLENARPROTOKOLL,17/31,18.03.2010,Plenarprotokoll vom 18.03.2010,"Plenarprotokoll 17/31\nDeutscher Bundestag\nStenografischer Bericht\n\n31. Sitzung\n\nBerlin, Donnerstag, den 18. März 2010\n\nI n h a l t :\nGlückwünsche zum Geburtstag der Abgeord-\nneten Dr. Cl..."


In [ ]:
'''
import xml.etree.ElementTree as et 

xtree = et.parse("MDB_STAMMDATEN.xml")
xroot = xtree.getroot()

df_cols = []
rows = []

for child in xroot:
    df_cols_ = df_cols.append(child.tag)
    rows_ = rows.append(child.text)

out_df = pd.DataFrame(rows, columns = df_cols)'''

In [ ]:
'''# read xml files
result = []
soup = BeautifulSoup(open("MDB_STAMMDATEN.xml", "r", encoding="utf-8"), "xml")
d = {}
for tag in soup.find_all(recursive=True):
    d[tag.name] = tag.get_text(strip=True)
    result.append(d)
        
#write in Dataframe
stammdaten = pd.DataFrame(result)'''

In [ ]:
'''stammdaten = stammdaten[:10]
stammdaten = stammdaten.drop_duplicates()
stammdaten = stammdaten["WAHLPERIODE" == "17"]'''

In [8]:
#read csv file of the names
gender_df = pd.read_csv("DIS25_liste_gender.csv",delimiter=";")

In [10]:
names = gender_df["Vorname_Nachname"].tolist()
partei = gender_df["Fraktion"].tolist()

# 2. Datentransformation

In [ ]:
'''
#extract the participants in columns "TEXT"
pattern_names = "Namensverzeichnis"
participants = []
for row in df["TEXT"]:
    if str(row).find(pattern_names):
        rowstr = str(row).split(pattern_names)
        participants_ = participants.append(rowstr[1].split("\n"))

#print(participants)
df["TEILNEHMER"] = participants '''

In [11]:
#split column "TEXT" and add only the important text into new column and the participants
conversation = []
pattern_start = "Beginn:"
pattern_end = "Schluss:"

for row in df["TEXT"]:
    rowstr = str(row).split(pattern_start)
    conversation.append(rowstr[1])

df["TEXT_SPLIT"] = conversation

In [12]:
df

,WAHLPERIODE,DOKUMENTART,NR,DATUM,TITEL,TEXT,TEXT_SPLIT
140,17,PLENARPROTOKOLL,17/22,10.02.2010,Plenarprotokoll vom 10.02.2010,"Plenarprotokoll 17/22\nDeutscher Bundestag\nStenografischer Bericht\n\n22. Sitzung\n\nBerlin, Mittwoch, den 10. Februar 2010\n\nI n h a l t :\nGlückwünsche zum Geburtstag des Abgeord-\nneten Leo D...","9.00 Uhr\nPräsident Dr. Norbert Lammert:\nDie Sitzung ist eröffnet.\n\nGuten Morgen, liebe Kolleginnen und Kollegen! Ich\nbegrüße Sie alle herzlich.\n\nGanz besonders herzlich begrüße ich den Kol..."
147,17,PLENARPROTOKOLL,17/23,24.02.2010,Plenarprotokoll vom 24.02.2010,"Plenarprotokoll 17/23\nDeutscher Bundestag\nStenografischer Bericht\n\n23. Sitzung\n\nBerlin, Mittwoch, den 24. Februar 2010\n\nI n h a l t :\nTagesordnungspunkt 1:\n\nBefragung der Bundesergierun...","13.00 Uhr\nVizepräsidentin Petra Pau:\nDie Sitzung ist eröffnet. Ich bitte Sie, Platz zu neh-\n\nmen, liebe Kolleginnen und Kollegen.\n\nIch rufe Tagesordnungspunkt 1 auf:\n\nBefragung der Bundes..."
154,17,PLENARPROTOKOLL,17/24,25.02.2010,Plenarprotokoll vom 25.02.2010,"Plenarprotokoll 17/24\nDeutscher Bundestag\nStenografischer Bericht\n\n24. Sitzung\n\nBerlin, Donnerstag, den 25. Februar 2010\n\nI n h a l t :\nErweiterung und Abwicklung der Tagesord-\nnung . ....","9.00 Uhr\nPräsident Dr. Norbert Lammert:\nDie Sitzung ist eröffnet. Nehmen Sie bitte Platz. Gu-\n\nten Morgen, liebe Kolleginnen und Kollegen.\n\nBevor ich Tagesordnungspunkt 3 aufrufe, teile ich..."
161,17,PLENARPROTOKOLL,17/25,26.02.2010,Plenarprotokoll vom 26.02.2010,"Plenarprotokoll 17/25\nDeutscher Bundestag\nStenografischer Bericht\n\n25. Sitzung\n\nBerlin, Freitag, den 26. Februar 2010\n\nI n h a l t :\nTagesordnungspunkt 18:\n\n– Beschlussempfehlung und Be...",9.00 Uhr\nPräsident Dr. Norbert Lammert:\nDie Sitzung ist eröffnet.\n\nGuten Morgen liebe Kolleginnen und Kollegen! Neh-\nmen Sie bitte Platz.\n\nIch rufe Tagesordnungspunkt 18 auf:\n\n– Beratung...
168,17,PLENARPROTOKOLL,17/26,03.03.2010,Plenarprotokoll vom 03.03.2010,"Plenarprotokoll 17/26\nDeutscher Bundestag\nStenografischer Bericht\n\n26. Sitzung\n\nBerlin, Mittwoch, den 3. März 2010\n\nI n h a l t :\nTagesordnungspunkt 1:\nBefragung der Bundesregierung: 13....","13.00 Uhr\nVizepräsidentin Petra Pau:\nDie Sitzung ist eröffnet. Ich begrüße Sie recht herz-\n\nlich, liebe Kolleginnen und Kollegen. Nehmen Sie bitte\nPlatz.\n\nIch rufe den Tagesordnungspunkt 1..."
175,17,PLENARPROTOKOLL,17/27,04.03.2010,Plenarprotokoll vom 04.03.2010,"Plenarprotokoll 17/27\nDeutscher Bundestag\nStenografischer Bericht\n\n27. Sitzung\n\nBerlin, Donnerstag, den 4. März 2010\n\nI n h a l t :\nWahl der Abgeordneten Cornelia Behm als\nstellvertreten...","9.00 Uhr\nPräsident Dr. Norbert Lammert:\nDie Sitzung ist eröffnet.\n\nNehmen Sie bitte Platz.  Guten Morgen, liebe Kolle-\nginnen und Kollegen! Ich begrüße Sie alle herzlich zu\ndieser Plenarsi..."
182,17,PLENARPROTOKOLL,17/28,05.03.2010,Plenarprotokoll vom 05.03.2010,"Plenarprotokoll 17/28\nDeutscher Bundestag\nStenografischer Bericht\n\n28. Sitzung\n\nBerlin, Freitag, den 5. März 2010\n\nI n h a l t :\nGlückwünsche zum Geburtstag der Abgeord-\nneten Ulla Lötze...","9.00 Uhr\nPräsident Dr. Norbert Lammert:\nDie Sitzung ist eröffnet. Guten Morgen, liebe Kolle-\n\nginnen und Kollegen! Ich begrüße Sie alle herzlich.\n\nDie Kollegin Ulla Lötzer feiert heute ihre..."
189,17,PLENARPROTOKOLL,17/29,16.03.2010,Plenarprotokoll vom 16.03.2010,"Plenarprotokoll 17/29\nDeutscher Bundestag\nStenografischer Bericht\n\n29. Sitzung\n\nBerlin, Dienstag, den 16. März 2010\n\nI n h a l t :\nGlückwünsche zum Geburtstag des Abgeord-\nneten Dr. Herb...","10.00 Uhr\nPräsident Dr. Norbert Lammert:\nDie Sitzung ist eröffnet. Nehmen Sie bitte Platz.\n\nGuten Morgen, liebe Kolleginnen und Kollegen! Ich\nbegrüße Sie alle herzlich zu den Haushaltsberatu..."
196,17,PLENARPROTOKOLL,17/30,17.03.2010,Plenarprotokoll vom 17.03.2010,"Plenarprotokoll 17/30\nDeutscher Bundestag\nStenografischer Bericht\n\n30. Sitzung\

In [ ]:
'''#split column "TEXT" and add only the important text into new column and the participants
conversation = []
participants = []
pattern_start = "Beginn:"
pattern_end = "Schluss:"
pattern_names = "Namensverzeichnis"

for row in df["TEXT"]:
    rowstr_start = str(row).split(pattern_start)
    rowstr_start1 = rowstr_start[1]
    rowstr_names = rowstr_start1.split(pattern_names)
    conversation.append(rowstr_names[0])
    participants.append(rowstr_names[1])

df["TEXT_SPLIT"] = conversation
df["TEILNEHMER"] = participants'''

In [13]:
#clean data
df["TEXT_SPLIT"] = df.apply(lambda row: re.sub("(\n)|(\r)|(\s+)", " ", row["TEXT_SPLIT"]), axis=1) #delete returns in data
df["TEXT_SPLIT"] = df.apply(lambda row: re.sub("-\s", "", row["TEXT_SPLIT"]), axis=1) #delete separate words with hyphen

In [14]:
df

,WAHLPERIODE,DOKUMENTART,NR,DATUM,TITEL,TEXT,TEXT_SPLIT
140,17,PLENARPROTOKOLL,17/22,10.02.2010,Plenarprotokoll vom 10.02.2010,"Plenarprotokoll 17/22\nDeutscher Bundestag\nStenografischer Bericht\n\n22. Sitzung\n\nBerlin, Mittwoch, den 10. Februar 2010\n\nI n h a l t :\nGlückwünsche zum Geburtstag des Abgeord-\nneten Leo D...","9.00 Uhr Präsident Dr. Norbert Lammert: Die Sitzung ist eröffnet. Guten Morgen, liebe Kolleginnen und Kollegen! Ich begrüße Sie alle herzlich. Ganz besonders herzlich begrüße ich den Kollegen L..."
147,17,PLENARPROTOKOLL,17/23,24.02.2010,Plenarprotokoll vom 24.02.2010,"Plenarprotokoll 17/23\nDeutscher Bundestag\nStenografischer Bericht\n\n23. Sitzung\n\nBerlin, Mittwoch, den 24. Februar 2010\n\nI n h a l t :\nTagesordnungspunkt 1:\n\nBefragung der Bundesergierun...","13.00 Uhr Vizepräsidentin Petra Pau: Die Sitzung ist eröffnet. Ich bitte Sie, Platz zu neh men, liebe Kolleginnen und Kollegen. Ich rufe Tagesordnungspunkt 1 auf: Befragung der Bundesregierung ..."
154,17,PLENARPROTOKOLL,17/24,25.02.2010,Plenarprotokoll vom 25.02.2010,"Plenarprotokoll 17/24\nDeutscher Bundestag\nStenografischer Bericht\n\n24. Sitzung\n\nBerlin, Donnerstag, den 25. Februar 2010\n\nI n h a l t :\nErweiterung und Abwicklung der Tagesord-\nnung . ....","9.00 Uhr Präsident Dr. Norbert Lammert: Die Sitzung ist eröffnet. Nehmen Sie bitte Platz. Gu ten Morgen, liebe Kolleginnen und Kollegen. Bevor ich Tagesordnungspunkt 3 aufrufe, teile ich Ihnen m..."
161,17,PLENARPROTOKOLL,17/25,26.02.2010,Plenarprotokoll vom 26.02.2010,"Plenarprotokoll 17/25\nDeutscher Bundestag\nStenografischer Bericht\n\n25. Sitzung\n\nBerlin, Freitag, den 26. Februar 2010\n\nI n h a l t :\nTagesordnungspunkt 18:\n\n– Beschlussempfehlung und Be...",9.00 Uhr Präsident Dr. Norbert Lammert: Die Sitzung ist eröffnet. Guten Morgen liebe Kolleginnen und Kollegen! Nehmen Sie bitte Platz. Ich rufe Tagesordnungspunkt 18 auf: – Beratung der Beschl...
168,17,PLENARPROTOKOLL,17/26,03.03.2010,Plenarprotokoll vom 03.03.2010,"Plenarprotokoll 17/26\nDeutscher Bundestag\nStenografischer Bericht\n\n26. Sitzung\n\nBerlin, Mittwoch, den 3. März 2010\n\nI n h a l t :\nTagesordnungspunkt 1:\nBefragung der Bundesregierung: 13....","13.00 Uhr Vizepräsidentin Petra Pau: Die Sitzung ist eröffnet. Ich begrüße Sie recht herz lich, liebe Kolleginnen und Kollegen. Nehmen Sie bitte Platz. Ich rufe den Tagesordnungspunkt 1 auf: Be..."
175,17,PLENARPROTOKOLL,17/27,04.03.2010,Plenarprotokoll vom 04.03.2010,"Plenarprotokoll 17/27\nDeutscher Bundestag\nStenografischer Bericht\n\n27. Sitzung\n\nBerlin, Donnerstag, den 4. März 2010\n\nI n h a l t :\nWahl der Abgeordneten Cornelia Behm als\nstellvertreten...","9.00 Uhr Präsident Dr. Norbert Lammert: Die Sitzung ist eröffnet. Nehmen Sie bitte Platz.  Guten Morgen, liebe Kolleginnen und Kollegen! Ich begrüße Sie alle herzlich zu dieser Plenarsitzung ve..."
182,17,PLENARPROTOKOLL,17/28,05.03.2010,Plenarprotokoll vom 05.03.2010,"Plenarprotokoll 17/28\nDeutscher Bundestag\nStenografischer Bericht\n\n28. Sitzung\n\nBerlin, Freitag, den 5. März 2010\n\nI n h a l t :\nGlückwünsche zum Geburtstag der Abgeord-\nneten Ulla Lötze...","9.00 Uhr Präsident Dr. Norbert Lammert: Die Sitzung ist eröffnet. Guten Morgen, liebe Kolle ginnen und Kollegen! Ich begrüße Sie alle herzlich. Die Kollegin Ulla Lötzer feiert heute ihren 60. Ge..."
189,17,PLENARPROTOKOLL,17/29,16.03.2010,Plenarprotokoll vom 16.03.2010,"Plenarprotokoll 17/29\nDeutscher Bundestag\nStenografischer Bericht\n\n29. Sitzung\n\nBerlin, Dienstag, den 16. März 2010\n\nI n h a l t :\nGlückwünsche zum Geburtstag des Abgeord-\nneten Dr. Herb...","10.00 Uhr Präsident Dr. Norbert Lammert: Die Sitzung ist eröffnet. Nehmen Sie bitte Platz. Guten Morgen, liebe Kolleginnen und Kollegen! Ich begrüße Sie alle herzlich zu den Haushaltsberatungen ..."
196,17,PLENARPROTOKOLL,17/30,17.03.2010,Plenarprotokoll vom 17.03.2010,"Plenarprotokoll 17/30\nDeutscher Bundestag\nStenografischer Bericht\n\n30. Sitzung\n\nBerlin, Mi

## Neues SubDataframe

In [ ]:
'''row = "11.00 Uhr Alterspräsident Dr. Heinz Riesenhuber: Guten Morgen, meine sehr verehrten Damen und Her ren! Liebe Kolleginnen und Kollegen, ich begrüße Sie zur konstituierenden Sitzung des 17. Deutschen Bundestags.  Parlamentarischer Brauch ist es – das entspricht unserer Geschäftsordnung; ich kann die Paragrafen zitieren –, dass der Älteste die erste Sitzung des Bundestags eröffnet. Ich bin am Sonntag, dem 1. Dezember 1935, geboren. Wenn jemand von den Kollegen im Saal älter ist als ich, dann spreche er jetzt oder er schweige für immer.  (Heiterkeit und Beifall) "
x = "Heinz Riesenhuber"

start = row.index(str(x) + ":")
end = row.index(')',start+1)
substring = row[start-1:end+1]
    

print(f"Start: {start}, End: {end}")
print(substring)'''

In [15]:
# write the components of speaker, speech and interruption
Sub = []

for row in df["TEXT_SPLIT"]:
    for i in names:
        x = str(i) + ":"
        if x in row:
            start = str(row).index(x)
            end = row.index(')',start+1)
            substring = row[start-1:end+1]
            Sub.append(substring)

In [47]:
#split the components and write into new dataframe
data = pd.DataFrame()
Redebeitrag = []
Person = []
Unterbrechung = []

for x in Sub:
    string = re.split(':', str(x), 1)
    string1 = re.split('\\(|\\)', str(string[1]))
    Person.append(string[0])
    Redebeitrag.append(string1[0])
    Unterbrechung.append(string1[1])

data["Redeinhalt"] = Redebeitrag
data["Vortragende_Person"] = Person
data["Unterbrechung"] = Unterbrechung

In [48]:
#drop columns with no relevant interruptions
data.drop(data[data["Unterbrechung"] == "A"].index, inplace = True)

In [49]:
data = data.loc[~data['Unterbrechung'].isin(partei)]

In [50]:
data = data[~data['Unterbrechung'].str.lower().str.contains("beifall")]

In [51]:
data

,Redeinhalt,Vortragende_Person,Unterbrechung
6,Wird die Bundesagentur für Arbeit aufgrund der nunmehr durch ein neues Verfahren möglichen Identifizierung bisher nicht zur Schwerbehindertenausgleichsabgabe zahlungsverpflichteter Arbeitgeber b...,Silvia Schmidt,Eisleben
9,"Die Sitzung ist eröffnet. Nehmen Sie bitte Platz. Gu ten Morgen, liebe Kolleginnen und Kollegen. Bevor ich Tagesordnungspunkt 3 aufrufe, teile ich Ihnen mit, dass es die interfraktionelle Verein...",Norbert Lammert,siehe 23. Sitzung
10,"Kollegin Möller, achten Sie bitte auf die Redezeit! Kornelia Möller",Petra Pau,DIE LINKE
13,Die Sitzung ist eröffnet. Guten Morgen liebe Kolleginnen und Kollegen! Nehmen Sie bitte Platz. Ich rufe Tagesordnungspunkt 18 auf: – Beratung der Beschlussempfehlung und des Berichts des Auswä...,Norbert Lammert,3. Ausschuss
14,Ich bitte um Entschuldigung für die kurze Unterbre chung und setze die Sitzung fort. Ich rufe die Tagesordnungspunkte 19 a und 19 b auf: a,Hermann Otto Solms,
19,"Wie ist der Stand der Simulation bezüglich der acht biophysikalischen Kriterien für die Neuabgrenzung der benachteiligten Gebiete, die von der Europäischen Kommission vorgeschlagen worden sind, ...",Michael Roth,Heringen
22,"Die Sitzung ist eröffnet. Nehmen Sie bitte Platz.  Guten Morgen, liebe Kolleginnen und Kollegen! Ich begrüße Sie alle herzlich zu dieser Plenarsitzung verbunden mit der Hoffnung, dass sie nicht...",Norbert Lammert,siehe 26. Sitzung
24,"Frau Kollegin, gestatten Sie eine Zwischenfrage des Kollegen Fricke? Dr. Gesine Lötzsch",Wolfgang Thierse,DIE LINKE
26,"Da Sie das gestatten, kann es jetzt geschehen. Klaus Ernst",Petra Pau,DIE LINKE
32,Ich eröffne die unterbrochene Sitzung wieder und teile die Ergebnisse der drei namentlichen Abstimmungen mit. Zunächst gebe ich das von den Schriftführerinnen und Schriftführern ermittelte Erge...,Wolfgang Thierse,D


# 3. Datenverarbeitung

In [ ]:
#case conversion & tokenizer
tokenizer = RegexpTokenizer(r'\w+')
data["TOKENS"] = data.apply (lambda row: tokenizer.tokenize(str(row["Unterbrechung"].lower())), axis=1)

In [ ]:
#stopwords removal
def remove_stopwords_from_list(list_in, language):
    return [a for a in list_in if a not in nltk.corpus.stopwords.words(language)]

In [ ]:
data["TOKENS"] = data.apply(lambda row: remove_stopwords_from_list(row["TOKENS"], "german"), axis=1)

In [ ]:
'''##doesn`t work
nlp = spacy.load("de_core_news_md")
nlp.max_length = 2000000

words = {}
for row in df["TOKENS"]:
    doc = nlp(str(row))
    for token in doc.ents:
        words = {token.text: token.label_}
        
nlp = spacy.load("de_core_news_md")
nlp.max_length = 2000000
liste = []
doc = nlp(str(df["TEXT_SPLIT"]))
liste.append([(X.text, X.label_) for X in doc.ents])
df["NER"] = liste'''

In [ ]:
'''#doesn't work
import nltk

from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.tree import Tree
nltk.download("punkt")
nltk.download('maxent_ne_chunker')
nltk.download('words')

text =
Alterspräsident Dr. Heinz Riesenhuber
und es wird weltweit geteilt. Die weltweite Gemein-
schaft, die Wissensgesellschaft bietet jedem offenen Zu-
griff auf alles, wenn er alles, was er hat, mit einbringt. Es
ist eine Gesellschaft, die aus Wissen lebt, die mit Wissen
wächst, die mit Wissen verantwortlich umzugehen ver-
steht, in der Wissen überall und jederzeit für jeden zu-
gänglich ist.

Es ist die Welt der Quanten, die wir langsam verste-
hen lernen  von der Vision des Quantencomputers bis
zur Nanotechnologie, wo im Allerkleinsten die Materie
andere Eigenschaften hat. Es ist die Welt der Gene, die
neue Möglichkeiten schafft. Nur ein Viertel der Krank-
heiten, die wir kennen, können wir an der Wurzel fassen.
Gentechnologie eröffnet die Chance, dass wir Krankhei-
ten heilen, dass wir menschliches Leid lindern, dass wir
gleichzeitig aus Wissen neue Märkte, neue Arbeitsplätze
und wirtschaftliches Wachstum schaffen.

Es ist die Welt der Computer, die schon jetzt unseren
Alltag beherrscht. Die Leistung der Chips verdoppelt
sich alle 18 Monate, und es halbiert sich ihr Preis. Da-
raus entstehen Internet und Unterhaltungselektronik.
Daraus entsteht eine Fülle von Möglichkeiten in Chemie
und im Maschinenbau. Daraus entsteht eine Fülle von
Möglichkeiten im Automobilbau. Daraus entstehen lau-
ter Arbeitsplätze, die für unsere Zukunft wichtig sind, in
einem Bereich, wo Deutschland immer stark war.
Deutschland war selten in den Spitzentechnologien
vorne, aber als Meister der Systeme wurden aus den ein-
zelnen Techniken neue Produkte geschaffen.

Der Chip ist Silizium, und Silizium ist Sand  und
Sand ist reichlich vorhanden. Und dazu braucht man
noch Intelligenz. Intelligenz, so sagt man uns, ist belie-
big vorhanden. In der Praxis findet man gelegentlich
Knappheiten.

(Heiterkeit bei Abgeordneten der CDU/CSU
und der FDP)

Aber sie wächst nach; es bleibt Hoffnung. Insofern ha-
ben wir hier die Chance eines Wachstums des guten Ge-
wissens, das die Wirklichkeit ändert und Zukunft für alle
schafft.

Dass diese Welt gelingt, hängt davon ab, ob wir die
Menschen haben, die sie gestalten, die Freude daran ha-
ben und die ihre Chance darin sehen. Wir werden uns
über vieles streiten, und wir haben uns über vieles ge-
stritten  manchmal mit intensiver Herzlichkeit. Aber
wir sind uns doch wohl weitestgehend darin einig, dass
Bildung und Forschung in diesen Jahren hohe Priorität
haben. Dafür braucht es nicht allein Geld  das auch ,
sondern auch Konzepte und unsere Bereitschaft, denen,
die hier gestalten, die Freude daran nicht zu nehmen.
Wir haben in diesen Jahren in einer Welt im Wandel die
Schulen umzubauen und aufzubauen. Wir haben gute
Schulen, und seit PISA sind sie noch besser geworden;
aber wir wissen, dass wir noch mehr erreichen können.

In der Föderalismusreform haben wir den Ländern die
Zuständigkeiten für die Bildung weitgehend zugeschrie-
ben. Sie wetteifern um das beste Schulsystem und um
die besten Chancen. Den Universitäten haben wir durch
einen Wettbewerb um Exzellenz, der über zehn Jahre an-
gelegt worden ist, den Ehrgeiz vermittelt, die Besten zu
sein und von jedem anderen zu lernen. Der Forschung
haben wir langfristig steigende Mittel zugesagt.

Was wir hier machen, ist eine Investition dafür, dass
wir stärker sind, wenn wir aus der Krise herauskommen.
Wir brauchen diese Stärke. Es gibt Aufgaben über das
Tagesgeschäft hinaus, die langfristig sind und in unter-
schiedlichen Bereichen angegangen werden müssen. Die
Menschen werden älter. Das ist ein Erfolg der Medizin.
Das ist ein Erfolg der Arbeitsbedingungen, die besser
geworden sind. Das ist ein Erfolg der Sozialsysteme.
Das ist also eine erfreuliche Sache und kein Problem.
Das Problem liegt darin, dass wir nicht rechtzeitig so
viele Kinder gezeugt haben, wie wir es hätten tun sollen.

(Heiterkeit)

Was hier und jetzt anzugehen ist,

(Renate Künast [BÜNDNIS 90/DIE GRÜ-
NEN]: Das wollen Sie jetzt angehen?  Heiter-
keit)

ist, dass wir in dieser Gesellschaft, die altert, die Kreati-
vität bewahren und das dritte Lebensalter nicht als ein
Zu-Ende-Leben des Lebens verstehen, sondern als neuen
Raum der Gestaltung mit einer Freiheit, die man über
das ganze frühere Leben nicht hatte. Man kann wählen;
das gilt sowohl für die Freizeit, das Ehrenamt und, wenn
man es will und kann, die Arbeit. Dies so zu organisie-
ren, dass die Menschen diese Chance ergreifen, dass sie
mitgestalten und aktiv dabei sind, ist eine unserer großen
Aufgaben.


nltk_results = ne_chunk(pos_tag(word_tokenize(text)))
for nltk_result in nltk_results:
    if type(nltk_result) == Tree:
        name = ''
        for nltk_result_leaf in nltk_result.leaves():
            name += nltk_result_leaf[0] + ' '
        print ('Type: ', nltk_result.label(), 'Name: ', name)'''

In [ ]:
#pos tagging
words = []
for row in data["TOKENS"]:
    tagged_tokens = nltk.pos_tag(row)
    words_ = words.append(tagged_tokens)

data["POS"] = words

In [ ]:
#lemmatizer
lemmatizer = WordNetLemmatizer()
data["LEMMATA"] = data.apply(lambda row: [lemmatizer.lemmatize(word) for word in row["TOKENS"]], axis=1)

In [ ]:
data

In [ ]:
'''#create new Dataframe and copy some old columns
data = df[["NR","DATUM","TEXT_SPLIT"]].copy()'''